In [21]:
# Imports (numpy, pandas, matplotlib, seaborn imported automatically)
from etherscan import Etherscan
from BondingCurveNexus import config

In [22]:
# activate etherscan client
eth_client = Etherscan(config.etherscan_api)

eth_price = float(eth_client.get_eth_last_price()['ethusd'])

In [23]:
eth_price

1731.71

# Investigate patterns in cover amounts and claim frequencies

## Bring in files and convert datetimes

In [25]:
# bring in csvs

# historic active cover amount
active_cover_amount = pd.read_csv('../raw_data/active_cover_amount.csv')
active_cover_amount.columns = ['date', 'USD', 'ETH']
active_cover_amount['date'] = pd.to_datetime(active_cover_amount['date']).dt.date

# claims history
claims_history = pd.read_csv('../raw_data/all_claims.csv')
claims_history['timestamp'] = pd.to_datetime(claims_history['timestamp'])

# cover history
cover_history = pd.read_csv('../raw_data/all_covers.csv')
cover_history['start_time'] = pd.to_datetime(cover_history['start_time']).dt.date
cover_history['end_time'] = pd.to_datetime(cover_history['end_time']).dt.date

## Investigate Distribution of Daily changes in Active cover amount

In [26]:
active_cover_amount = active_cover_amount.groupby('date').max()

In [27]:
active_cover_amount['ETH_change_perc'] = active_cover_amount['ETH'] / active_cover_amount['ETH'].shift(1) - 1
active_cover_amount['ETH_change_val'] = active_cover_amount['ETH'] - active_cover_amount['ETH'].shift(1)

In [28]:
active_cover_amount

,USD,ETH,ETH_change_perc,ETH_change_val
date,,,,
2019-07-12,9.775000e+02,3.549383,NaN,NaN
2019-07-13,2.807950e+04,103.606745,28.190074,100.057362
2019-07-14,7.119130e+04,304.197325,1.936076,200.590580
2019-07-15,1.085736e+05,472.018185,0.551684,167.820860
2019-07-16,2.327708e+05,1161.878556,1.461512,689.860371
...,...,...,...,...
2022-06-06,3.219952e+08,172156.440888,-0.006095,-1055.660376
2022-06-07,2.976967e+08,167008.480992,-0.029903,-5147.959896
2022-06-08,3.012918e+08,168740.869302,0.010373,1732.388310


In [32]:
active_cover_amount[-240:]

,USD,ETH,ETH_change_perc,ETH_change_val
date,,,,
2021-10-14,4.316843e+08,114637.423597,-0.065447,-8028.045501
2021-10-15,4.637110e+08,119904.621381,0.045947,5267.197784
2021-10-16,4.721258e+08,123408.207362,0.029220,3503.585981
2021-10-17,4.784494e+08,123726.900447,0.002582,318.693085
2021-10-18,4.765855e+08,124704.274779,0.007899,977.374332
...,...,...,...,...
2022-06-06,3.219952e+08,172156.440888,-0.006095,-1055.660376
2022-06-07,2.976967e+08,167008.480992,-0.029903,-5147.959896
2022-06-08,3.012918e+08,168740.869302,0.010373,1732.388310


In [29]:
active_cover_amount[-240:]['ETH_change_val'].describe()

count      240.000000
mean       230.231857
std       4747.128874
min     -19091.121391
25%      -1501.448847
50%         15.189586
75%       2625.313310
max      16371.068444
Name: ETH_change_val, dtype: float64

## Investigate premium income & cover drop-off

In [30]:
cover_history

,block_number,start_time,premium,address,cover_id,end_time,currency,amount,project,amount_usd,premium_usd
0,8667271,2019-10-03,0.012991,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,84,2020-10-02,ETH,1.0,Uniswap v1,1721.14,22.359505
1,8759426,2019-10-17,0.064066,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,93,2020-03-15,ETH,12.0,Uniswap v1,20653.68,110.266053
2,8873386,2019-11-04,0.064066,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,98,2020-04-02,ETH,12.0,Uniswap v1,20653.68,110.266053
3,8895454,2019-11-08,0.074743,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,100,2020-04-06,ETH,14.0,Uniswap v1,24095.96,128.643729
4,8935109,2019-11-14,0.172977,0xc0a47dfe034b400b47bdad5fecda2621de6c4d95,106,2020-02-12,ETH,54.0,Uniswap v1,92941.56,297.718344
...,...,...,...,...,...,...,...,...,...,...,...
7462,12246810,2021-04-15,448.459959,0xcb876f60399897db24058b2d58d0b9f713175eef,3901,2021-07-14,DAI,70000.0,Pool Together v3,69979.00,448.325421
7463,11785120,2021-02-03,1.127023,0xa4c8d221d8bb851f83aadd0223a8900a6921a349,3150,2022-02-03,ETH,7.0,Set Protocol v2,12047.98,1939.764526
7464,11812010,2021-02-07,4.015933,0xa4c8d221d8bb851f83aadd0223a8900a6921a349,3198,2022-02-07,ETH,25.0,Set Protocol v2,43028.50,6911.982535
7465,11965147,2021-03-03,0.134255,0xa4c8d221d8bb851f83aadd0223a8900a6921a349,3531,2021-06-01,ETH,10.0,Set Protocol v2,17211.40,231.071350


In [31]:
cover_history['premium_eth'] = np.where(cover_history['currency'] == 'DAI',
                                         cover_history['premium']/eth_price, cover_history['premium'])
cover_history['amount_eth'] = np.where(cover_history['currency'] == 'DAI',
                                         cover_history['amount']/eth_price, cover_history['amount'])

In [44]:
cover_history['rate'] = cover_history['premium_eth']/cover_history['amount_eth']

In [45]:
cover_history['rate'].describe()

count    7467.000000
mean        0.009966
std         0.014127
min         0.001068
25%         0.002136
50%         0.006407
75%         0.012813
max         0.447375
Name: rate, dtype: float64

### Premium & Amount History (since June 2020)

In [47]:
# group by day
premium_history = cover_history.groupby('start_time')[['premium_eth']].sum().reset_index()
amount_history = cover_history.groupby('start_time')[['amount_eth']].sum().reset_index()

In [48]:
premium_history = premium_history.iloc[153:]
amount_history = amount_history.iloc[153:]

In [49]:
premium_history = premium_history.set_index('start_time')
amount_history = amount_history.set_index('start_time')

In [50]:
# fill in empty dates with no cover buys
idx = pd.date_range('2020-06-01', '2022-06-10')
premium_history = premium_history.reindex(idx, fill_value=0)
amount_history = amount_history.reindex(idx, fill_value=0)

In [51]:
premium_history.describe()

,premium_eth
count,740.000000
mean,20.651142
std,140.459813
min,0.000000
25%,1.941638
50%,5.861952
75%,14.244101
max,3704.850490


In [75]:
amount_history[500:].describe()

,amount_eth
count,240.000000
mean,3092.239315
std,4134.571986
min,0.000000
25%,564.828909
50%,1449.982774
75%,4048.486121
max,23060.799545


### Drop-off in cover amounts (since 14 Oct 2021)


In [67]:
# group by day
drop_history = cover_history.groupby('end_time')[['amount_eth']].sum().reset_index()
drop_history = drop_history.iloc[569:807]
drop_history = drop_history.set_index('end_time')
idx = pd.date_range('2021-10-14', '2022-06-10')
drop_history = drop_history.reindex(idx, fill_value=0)

In [68]:
drop_history

,amount_eth
2021-10-14,1067.280336
2021-10-15,186.866606
2021-10-16,934.125535
2021-10-17,301.288605
2021-10-18,1031.210919
...,...
2022-06-06,12688.547205
2022-06-07,197.000000
2022-06-08,791.125535
2022-06-09,583.845009


In [69]:
drop_history.mean()

amount_eth    2938.493783
dtype: float64

## Investigate frequency and size of accepted claims

In [14]:
# filter accepted claims only
accepted_claims = claims_history[claims_history['verdict'] == 'Accepted']

# work out days since first claim
days_since_first_claim = (pd.Timestamp.now() - accepted_claims['timestamp'].iloc[0]).days

# count number of claims and divide by number of days
avg_claim_freq = accepted_claims['verdict'].count() / days_since_first_claim

avg_claim_freq 

0.03155339805825243

In [15]:
accepted_claims['amount_eth'] = np.where(accepted_claims['currency'] == 'DAI',
                                         accepted_claims['amount']/eth_price, accepted_claims['amount'])

/tmp/ipykernel_615/834324826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted_claims['amount_eth'] = np.where(accepted_claims['currency'] == 'DAI',


In [16]:
accepted_claims

,timestamp,cover_id,block_number,verdict,claim_id,project,amount_usd,amount,currency,start_time,amount_eth
0,2020-02-18 17:01:18,152,9508333,Accepted,2,bZx v1,8050.76,4.0,ETH,2020-02-03 19:58:33,4.000000
2,2020-02-19 23:55:22,158,9516760,Accepted,5,bZx v1,2598.70,2600.0,DAI,2020-02-05 01:27:47,1.283570
3,2020-02-18 17:07:50,121,9508364,Accepted,3,bZx v1,29985.00,30000.0,DAI,2019-12-10 15:49:23,14.810427
69,2021-02-04 23:20:43,729,11792745,Accepted,72,Yearn Finance (all vaults),12076.14,6.0,ETH,2020-08-24 17:36:23,6.000000
70,2021-02-05 02:47:32,278,11793668,Accepted,73,Yearn Finance (all vaults),1399.30,1400.0,DAI,2020-04-16 00:45:45,0.691153
72,2021-02-05 02:49:38,616,11793678,Accepted,74,Yearn Finance (all vaults),48304.56,24.0,ETH,2020-08-16 10:02:18,24.000000
73,2021-02-05 03:19:09,203,11793813,Accepted,75,Yearn Finance (all vaults),8995.50,9000.0,DAI,2020-02-20 04:33:53,4.443128
74,2021-02-05 17:42:46,652,11797703,Accepted,77,Yearn Finance (all vaults),12076.14,6.0,ETH,2020-08-17 21:28:10,6.000000
75,2021-02-05 18:58:03,1815,11798055,Accepted,78,Yearn Finance (all vaults),20126.90,10.0,ETH,2020-09-16 23:08:00,10.000000
76,2021-02-06 07:31:02,759,11801426,Accepted,80,Yearn Finance (all vaults),100634.50,50.0,ETH,2020-08-28 06:40:35,50.000000


In [17]:
accepted_claims.amount_eth.describe()

count      26.000000
mean       60.219984
std       193.485521
min         0.691153
25%         5.202607
50%        14.810427
75%        31.815265
max      1000.000000
Name: amount_eth, dtype: float64